# Scrap repo stats using multiprocessing

### Author: Crystal Zang

This notebook utilized GitHub access tokens (PAT) to scrape GitHub repository statistics such as stargazers, watchers, forks, and topics. One PAT would scrape at a rate of 5000 repositories per hour. Utilizing 36 PATs we would scrape 10,288,063 repositories in about XXX hours at a rate of 15,514 repositories per hour.

#### Warnings
You should not commit any access topen to GitHub, which would result in access token being revoked.


In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
# load packages 
import os
import psycopg2 as pg
from sqlalchemy import create_engine
import pandas as pd
import requests as r
import string 
import json
import base64
import urllib.request
import itertools 
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from github import Github, RateLimitExceededException, BadCredentialsException, BadAttributeException, GithubException, UnknownObjectException, BadUserAgentException
import warnings
import datetime

import multiprocessing
#from multiprocessing.pool import ThreadPool as Pool
from multiprocessing import Pool, freeze_support

import concurrent.futures

warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
#os.environ['db_user'] = ''
#os.environ['db_pwd'] = ''

# connect to the database, download data, limit to repos with at least 20,000 commits?
connection = pg.connect(host = 'postgis1', database = 'sdad', 
                        user = os.environ.get('db_user'), 
                        password = os.environ.get('db_pwd'))

raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked WHERE (commits BETWEEN '700' AND '800')'''
#raw_slug_data = '''SELECT * FROM gh_2007_2020.repos_ranked WHERE commits < 1000'''

# convert to a dataframe, show how many missing we have (none)
raw_slug_data = pd.read_sql_query(raw_slug_data, con=connection)
print(raw_slug_data.head())
print(raw_slug_data.shape)
print(raw_slug_data.isna().sum())

#get rid of leading and ending space, save slugs to a list
raw_slugs = raw_slug_data["slug"].tolist()
slugs = []
for s in raw_slugs:
    slugs.append(s.strip())  
print(len(slugs))
print(slugs[0], slugs[len(slugs)-1])

#PATs access token, saved as a dataframe
github_pats = '''SELECT * FROM gh_2007_2020.pats_update'''
github_pats = pd.read_sql_query(github_pats, con=connection)

#PATs access token, saved as a list
access_tokens = github_pats["token"]

#number of tokens available for use, a numeric value
num_token = '''SELECT COUNT(*) FROM gh_2007_2020.pats_update'''
num_token = pd.read_sql_query(num_token, con=connection)
num_token=num_token.iloc[0]['count']

connection.close()

                                 id        spdx  \
0  MDEwOlJlcG9zaXRvcnk5Mjc0MDI1Ng==     GPL-3.0   
1  MDEwOlJlcG9zaXRvcnk2NTM3MjgwMA==  Apache-2.0   
2  MDEwOlJlcG9zaXRvcnk5NDQ0ODI4NQ==     GPL-2.0   
3      MDEwOlJlcG9zaXRvcnkzNTAyMTIw         MIT   
4      MDEwOlJlcG9zaXRvcnk1OTk4NzU4         MIT   

                                    slug           createdat  \
0                halvors/Nuclear-Physics 2017-05-29 12:59:30   
1  pushtechnology/diffusion-rest-adapter 2016-08-10 10:16:10   
2      ezsystems/developer-documentation 2017-06-15 14:36:36   
3                  scottjehl/picturefill 2012-02-21 08:35:14   
4        DenTelezhkin/DTTableViewManager 2012-09-28 14:56:08   

                                         description primarylanguage  \
0  Nuclear Physics is a mod that brings in realis...            Java   
1                             Diffusion REST Adapter            Java   
2  Source for the developer documentation for eZ ...            HTML   
3  A responsive imag

In [7]:
# index ranges from 0 to maximum number of PATs available
def get_access_token(github_pat_index):
    if github_pat_index < num_token:
       # print("Extracting access token #", github_pat_index+1,", total", num_token, "tokens are available.")
        return github_pats.token[github_pat_index]
    else:
        print("token exceed limit")

## Function to scrape one slug using specified pat index

In [8]:
#global github_pat_index
def pull_repo_stats(slug, github_pat_index):
    if github_pat_index >= len(access_tokens):
        github_pat_index -= len(access_tokens)
        print(f"***Pat access token exceed limit, restart access token loop with #", github_pat_index)  

    access_token = get_access_token(github_pat_index)
    #print("Scrapping --", slug,". Extracting access token #", github_pat_index+1,", total", num_token, "tokens are available.")
    #if false, retry until true, max number of retry is set to 3 times
    #g = Github(access_token, retry = 3, timeout = 15)
    g = Github(access_token)
    repo = g.get_repo(slug)
    stars = repo.stargazers_count
    watchers = repo.subscribers_count
    forks = repo.forks_count
    topics = repo.get_topics()

    results = (slug, stars, watchers, forks, topics)
    return results

In [9]:
def get_result(results):
    global slug_log
    global stars_log
    global watchers_log
    global forks_log
    global topics_log
    if results is None:
        slug_log.append(None)
        stars_log.append(None)
        watchers_log.append(None)
        forks_log.append(None)
        topics_log.append(None)  
    else:
        slug_log.append(results[0])
        stars_log.append(results[1])
        watchers_log.append(results[2])
        forks_log.append(results[3])
        topics_log.append(results[4])
       # print("appending results")
    
def get_stars(stars):
    global stars_log
    stars_log.append(stars)
    #print("appending results")

In [23]:
# Example
slugs_example = ["moderndive/ModernDive_book", "DSPG-Young-Scholars-Program/dspg21oss", 
                 "unknownrepo", "moderndive/ModernDive_book", "lsst/ip_diffim", "esrlabs/chipmunk", "paulmillr/chokidar"]

i = 1
print(slugs_example[i])
result = pull_repo_stats(slugs_example[i], 0)
print(result[0])

slug_log = []
stars_log = []
watchers_log = []
forks_log = []
topics_log = []

get_result(result)
print(slug_log)
#print(result[0])
#print(result[1] ==None)

DSPG-Young-Scholars-Program/dspg21oss
DSPG-Young-Scholars-Program/dspg21oss
['DSPG-Young-Scholars-Program/dspg21oss']


In [21]:
slugs_try = slugs[0:3000]
print(slugs_try)
print(len(slugs_try))
print(slugs_try[0], slugs_try[2999])

['halvors/Nuclear-Physics', 'pushtechnology/diffusion-rest-adapter', 'ezsystems/developer-documentation', 'scottjehl/picturefill', 'DenTelezhkin/DTTableViewManager', 'podio/podio-objc', 'netgen-layouts/layouts-sylius-site', 'ribasco/ucgdisplay', 'vimeo/graph-explorer', 'jpuri/react-draft-wysiwyg', 'alkacon/alkacon-oamp', 'evpobr/MyHomeLib', 'frndxyz/unomp-windows', 'prism-em/prismatic', 'ahoskins/Winston', 'surfliner/surfliner-mirror', 'rezariadi21/reza', 'nWidart/laravel-modules', 'Faris90/Gancer', 'EarnGold/Laravel-Module-Management', 'yinshuangfei/programming-book-zh_CN', 'LineageOS/tools_repo', 'rwilson-sfi/JanitorMonkey', 'OverlayPHP/Overlay-Laravel-Modules', 'orocos-toolchain/log4cpp', 'DavidBabel/zz', 'nsathiya/FbPageManager', 'novikov-school/NSpectator', 'medikoo/deferred', 'emacs-evil/evil-collection', 'KepaCantero/HBPCLE', 'radhakrishnaakamat/Interview', 'delving/hub3', 'BRI-EES-House/Logic-EES-WEB', 'pas-jim/crtmpserver-spoon', 'HDI-Project/ballet', 'bnoordhuis/ragel', 'aaaw

## Multiprocessing approach: pool.apply_async for multiple parameters

In [17]:
# Approach 2. pool.apply_async for multiple parameters
github_pat_index = None
if __name__ == '__main__':
    pool = multiprocessing.Pool(10) # using 10 cores for scrapping, leave 6 cores to run other stuff
    
    slug_log = []
    stars_log = []
    watchers_log = []
    forks_log = []
    topics_log = []

    start_time = datetime.datetime.now()
    print("Start scraping. Start time:", start_time)
    github_pat_index = 0
    #print("1. outer loop pat index", github_pat_index)
    
    for slug in slugs_try:
        #github_pat_index = 0
        if github_pat_index >= len(access_tokens):
            github_pat_index -= len(access_tokens)
            print("***Pat access token exceed limit, restart access token loop with #", github_pat_index)
        while True:
            try:
                #github_pat_index = 3
               # print("2. innner loop pat index", github_pat_index)      
                pool.apply_async(pull_repo_stats, args=(slug,  github_pat_index),callback = get_result)        
                print("~~~~~~~~~~~~~~~slug: ",slug, "using access token #", github_pat_index, "~~~~~~~~~~~~~~~~")
                print(slug_log)
                break 
            except RateLimitExceededException as e:
                print(e.status)
                print('WARNING: Rate limit exceeded --', slug, ", using access token #", github_pat_index)
                #time.sleep(300)
                github_pat_index+=1
                print("***Exit current access token, proceed with next aceess token #", github_pat_index, "rescrape --",slug)
                break
            except BadCredentialsException as e:
                print(e.status)
                print('WARNING: Bad credentials exception --', slug, ", using access token #", github_pat_index)
                github_pat_index+=1
                print("***Exit current access token, proceed with next aceess token #", github_pat_index, "rescrape --",slug)
                break
            except UnknownObjectException as e:
                print(e.status)
                print('WARNING: Unknown object exception --', slug)
                break
            except GithubException as e:
                print(e.status)
                print('General exception --', slug)
                break
            except r.exceptions.ConnectionError as e:
                print('Retries limit exceeded --', slug)
                print(str(e))
                time.sleep(10)
                continue
            except r.exceptions.Timeout as e:
                print('Time out exception --', slug)
                print(str(e))
                time.sleep(10)
                continue
    pool.close()
    pool.join()
    
    print(stars_log)
    final_log = pd.DataFrame({'slug': slug_log, "stars": stars_log, 'watchers': watchers_log, 'forks': forks_log, 'topics': topics_log}, columns=["slug", "stars", "watchers", "forks", "topics"])

    end_time = datetime.datetime.now()
    print("Finished scraping", len(final_log), "of", len(slugs_try), "records at", end_time)
    print("It took", end_time-start_time, "to run.")

Start scraping. Start time: 2021-07-02 14:31:13.346664
~~~~~~~~~~~~~~~slug:  halvors/Nuclear-Physics using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  pushtechnology/diffusion-rest-adapter using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  ezsystems/developer-documentation using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  scottjehl/picturefill using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  DenTelezhkin/DTTableViewManager using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  podio/podio-objc using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  netgen-layouts/layouts-sylius-site using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  ribasco/ucgdisplay using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  vimeo/graph-explorer using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  jpuri/react-draft-wysiwyg using access token # 0 ~~~~~~~~~~~~~~~~
[]
~~~~~~~~~~~~~~~slug:  alkac

In [18]:
slug_log 

['scottjehl/picturefill',
 'pushtechnology/diffusion-rest-adapter',
 'ribasco/ucgdisplay',
 'jpuri/react-draft-wysiwyg',
 'halvors/Nuclear-Physics',
 'ezsystems/developer-documentation',
 'podio/podio-objc',
 'vimeo/graph-explorer',
 'evpobr/MyHomeLib',
 'prism-em/prismatic',
 'ahoskins/Winston',
 'frndxyz/unomp-windows',
 'nWidart/laravel-modules',
 'rezariadi21/reza',
 'surfliner/surfliner-mirror',
 'Faris90/Gancer',
 'yinshuangfei/programming-book-zh_CN',
 'DenTelezhkin/DTTableViewManager',
 'LineageOS/tools_repo',
 'rwilson-sfi/JanitorMonkey',
 'OverlayPHP/Overlay-Laravel-Modules',
 'EarnGold/Laravel-Module-Management',
 'alkacon/alkacon-oamp',
 'orocos-toolchain/log4cpp',
 'DavidBabel/zz',
 'medikoo/deferred',
 'nsathiya/FbPageManager',
 'novikov-school/NSpectator',
 'emacs-evil/evil-collection',
 'delving/hub3',
 'radhakrishnaakamat/Interview',
 'bnoordhuis/ragel',
 'DigitalCampus/maf-oppia-mobile',
 'KepaCantero/HBPCLE',
 'zmjack/Dawnx',
 'HDI-Project/ballet',
 'BRI-EES-House/Lo

In [15]:
forks_log

[295]

In [19]:
topics_log

[[],
 [],
 ['graphics',
  'display',
  'lcd',
  'hd44780',
  'oled',
  'i2c',
  'spi',
  'parallel',
  'raspberry-pi',
  'arm',
  'embedded-systems',
  'library',
  'u8g2',
  'java',
  '8-bit',
  'iot',
  'e-ink',
  'single-board-computers',
  'glcd'],
 ['wysiwyg-editor', 'draft-js', 'reactjs', 'javascript'],
 ['minecraft',
  'minecraft-mod',
  'minecraft-forge',
  'minecraft-forge-mod',
  'atomic-science',
  'resonant-induction',
  'resonant',
  'induction',
  'electrodynamics',
  'forge',
  'edx',
  'nuclearphysics'],
 ['documentation', 'ezplatform'],
 [],
 [],
 [],
 [],
 ['schedule-generator', 'angular', 'university'],
 [],
 ['laravel-modules', 'laravel', 'architecture', 'php', 'hacktoberfest'],
 [],
 [],
 [],
 [],
 ['ios', 'uitableview', 'protocol', 'swift', 'datasource', 'delegate'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['testing',
  'ndepend',
  'jetbrains',
  'xspec-family',
  'nunit',
  'agile',
  'rspec',
  'nspec',
  'cucumber'],
 ['emacs', 'evil', 'evil-mode', 'bind

In [20]:
#print(final_log)
final_log

,slug,stars,watchers,forks,topics
0,scottjehl/picturefill,10069,367,1136,[]
1,pushtechnology/diffusion-rest-adapter,2,21,0,[]
2,ribasco/ucgdisplay,11,3,2,"[graphics, display, lcd, hd44780, oled, i2c, s..."
3,jpuri/react-draft-wysiwyg,5131,88,928,"[wysiwyg-editor, draft-js, reactjs, javascript]"
4,halvors/Nuclear-Physics,10,5,10,"[minecraft, minecraft-mod, minecraft-forge, mi..."
...,...,...,...,...,...
975,theWhiteFox/theWhiteFox,2,1,1,"[html5, javascript, jquery, gulp, npm, css3, s..."
976,balena-os/balena-intel,18,9,13,[]
977,clearlinux/common,30,8,16,"[integration-framework, distro-tools, rpm-buil..."
978,Esri/military-tools-geoprocessing-toolbox,24,21,8,"[military-tools-for-arcgis, arcgis-solutions, ..."


In [31]:
# read in the file and check
print(final_log.head())
print(final_log)
print(final_log.isna().sum())
print(final_log.shape)

                        slug  stars  watchers  forks topics
0              O330oei/Linxx      0         1      0     []
1              telenub/tgbot      0         1      0     []
2          lazykillu/userbot      0         1      0     []
3          Declan57/botkulah      0         1      0     []
4  sukhpreet1427/darkxusrbot      0         1      0     []
                               slug  stars  watchers  forks topics
0                     O330oei/Linxx      0         1      0     []
1                     telenub/tgbot      0         1      0     []
2                 lazykillu/userbot      0         1      0     []
3                 Declan57/botkulah      0         1      0     []
4         sukhpreet1427/darkxusrbot      0         1      0     []
..                              ...    ...       ...    ...    ...
94           kazuyouaoki/munish-dev      0         1      0     []
95                    YueLinHo/serf      0         0      0     []
96  FruitCrushSaga/Fruit-Crush-Saga  